---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="66cf1ca3" -->
# Table des matières
1. [Séparer ses données en combien d'ensembles?](#séparer-ses-données-en-combien-densembles)
    1. [Un ensemble d'entraînement?](#un-ensemble-dentraînement)
    1. [Un ensemble d'entraînement et un de validation?](#un-ensemble-dentraînement-et-un-de-validation)
    1. [Un ensemble d'entraînement, un de validation, et un de test?](#un-ensemble-dentraînement-un-de-validation-et-un-de-test)
1. [Principes de la validation croisée](#principes-de-la-validation-croisée)
  1. [Entraînement d'un classificateur](#entraînement-dun-classificateur)
    1. [Raison 1](#raison-1)
    1. [Raison 2](#raison-2)
    1. [Raison 3](#raison-3)
  1. [Entraînement de multiples classificateurs et sélection du meilleur](#entraînement-de-multiples-classificateurs-et-sélection-du-meilleur)
1. [Un exemple](#un-exemple)
  1. [Lecture et préparation des données](#lecture-et-préparation-des-données)
      1. [Balancement des classes](#balancement-des-classes)
      1. [Normalisation des données](#normalisation-des-données)
  1. [Classification par forêt aléatoire en utilisant les paramètres par défaut](#classification-par-forêt-aléatoire-en-utilisant-les-paramètres-par-défaut)
  1. [Optimisation des hyperparamètres du classificateur par forêt aléatoire](#optimisation-des-hyperparamètres-du-classificateur-par-forêt-aléatoire)
1. [Comment choisir son modèle final?](#comment-choisir-son-modèle-final)
    1. [Optimisation sur grille et affichage des résultats](#optimisation-sur-grille-et-affichage-des-résultats)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import warnings

import imblearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from imblearn.over_sampling import SMOTE
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import StandardScaler

sns.set(color_codes=True)


warnings.filterwarnings("ignore")


seed = 42
np.random.seed(seed)



<!-- #region id="4f442694" -->
Dans ce module, nous allons discuter de l'importance de la séparation des données lors d'un projet
en apprentissage automatique. La validation croisée est une méthode particulièrement efficace qui implémente cette idée.

Nous allons montrer comment la validation croisée peut être utilisée afin d'optimiser les performances des algorithmes
de traitement de données (régression, classification, etc.) Nous allons toutefois nous limiter
aux applications en classification dans ce module. Nous allons également montrer comment utiliser la validation croisée
pour faire la sélection de modèles. C'est-à-dire, pour choisir le meilleur parmi plusieurs classificateurs disponibles.
Cela est très utile, car il existe au moins une dizaine de familles de classificateurs, dont chacune a ses variantes.

> À noter que les résultats numériques affichés pour le classificateur de type forêt aléatoire peuvent varier d'une
> exécution du tutoriel à la suivante. Toutefois, les conclusions que l'on tire des résultats affichés demeurent les mêmes.
<!-- #endregion -->

<!-- #region id="703a5642" -->
# <a id=séparer-ses-données-en-combien-densembles>Séparer ses données en combien d'ensembles?</a>
<!-- #endregion -->

<!-- #region id="bfc952aa" -->
Imaginons qu'un professeur veuille faire passer un examen à ses étudiants. Pour les entraînements, il leur fournit
une liste de 100 problèmes à résoudre avec la solution connue. Il prévoit les tester en leur posant 10 questions.
<!-- #endregion -->

<!-- #region id="fb5953ec" -->
## <a id=un-ensemble-dentraînement>Un ensemble d'entraînement?</a>
<!-- #endregion -->

<!-- #region id="d57facd4" -->
Dans ce cas-ci, les étudiants s'entraînent avec la liste et sont testés avec 10 questions de la même liste. S'ils
ont bien mémorisé la série de questions-réponses, ils peuvent avoir $100~\%$ à l'examen; pas
besoin de comprendre la matière du cours. C'est du surapprentissage.
<!-- #endregion -->

<!-- #region id="176eb22c" -->
## <a id=un-ensemble-dentraînement-et-un-de-validation>Un ensemble d'entraînement et un de validation?</a>
<!-- #endregion -->

<!-- #region id="bb4af0ef" -->
Dans ce cas-ci, les étudiants s'entraînent avec la liste et sont testés avec 10 nouvelles questions
qu'ils n'ont jamais vues. Seuls ceux qui ont compris la matière avec la liste d'entraînement
passeront l'étape de validation. Peu probable qu'ils atteignent le $100~\%$. Cette étape nous assure qu'ils savent
résoudre des problèmes.
<!-- #endregion -->

<!-- #region id="a6365452" -->
## <a id=un-ensemble-dentraînement-un-de-validation-et-un-de-test>Un ensemble d'entraînement, un de validation, et un de test?</a>
<!-- #endregion -->

<!-- #region id="4d94e5ce" -->
Lequel parmi les étudiants ayant complété l'étape précédente est le meilleur? Plusieurs étudiants ont
des résultats assez similaires. Se pourrait-il que les 10 questions du professeur aient été biaisées d'une manière
ou d'une autre (style d'enseignement particulier?), ce qui aurait affecté les résultats du test? On demande alors
à un autre professeur enseignant la même matière de fournir un dernier ensemble de nouvelles questions,
de même niveau, afin de sélectionner le meilleur étudiant. Cette approche est utilisée afin d'évaluer les
performances d'un grand nombre d'étudiants au niveau provincial plutôt que local.

En résumé:

- deux ensembles de données sont nécessaires pour bien entraîner un modèle,
- mais trois ensembles sont nécessaires pour bien entraîner **et** sélectionner le meilleur modèle.
<!-- #endregion -->

<!-- #region id="ef01fc84" -->
# <a id=principes-de-la-validation-croisée>Principes de la validation croisée</a>
<!-- #endregion -->

<!-- #region id="651d7946" -->
Nous allons voir comment l'utiliser pour entraîner un ou plusieurs modèles, afin de sélectionner le meilleur d'entre eux.
<!-- #endregion -->

<!-- #region id="9c90f3f5" -->
## <a id=entraînement-dun-classificateur>Entraînement d'un classificateur</a>
<!-- #endregion -->

<!-- #region id="eb838600" -->
La figure suivante montre l'approche couramment utilisée pour l'entraînement d'un modèle. Cet exemple utilise
une validation croisée à cinq plis (*Folds*); le nombre de plis pourrait être différent.

On sépare initialement
les données en un ensemble d'entraînement (*Training Dataset*) et un autre de test (*Testing Dataset*).
Puis l'ensemble d'entraînement est subdivisé à son tour en plusieurs séries de plis. Par convention, dans chaque pli, les données en bleu
correspondent à l'ensemble de validation, et les données en vert à l'ensemble d'entraînement.
<!-- #endregion -->

<!-- #region id="9087f399" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/training-folds.png"  width="500" />
    <div>
    <font size="0.5">Image Source: https://tex.stackexchange.com/questions/492892/drawing-three-tables-with-tikz/</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="595dc6cd" -->
Les données d'entraînement sont utilisées pour estimer les valeurs des paramètres
d'un classificateur. Les données de test sont utilisées pour calculer la métrique
finale (exactitude, précision, rappel, etc.) du classificateur sur de nouvelles données jamais vues par le classifieur.

Pourquoi utiliser des plis pour l’entraînement du modèle? Il y a au moins trois bonnes raisons à cela.
<!-- #endregion -->

<!-- #region id="02c047a7" -->
### <a id=raison-1>Raison 1</a>
<!-- #endregion -->

<!-- #region id="36af7dfe" -->
Il arrive que le nombre de données d'entraînement soit limité.
Si l'on sépare l'ensemble d'entraînement en $80~\%$ train et $20~\%$ validation, on obtient une valeur de métrique pouvant être affectée par le faible nombre de données. Il est préférable d'utiliser cinq échantillonnages
différents comme sur la figure précédente, puis d’entraîner et tester le modèle sur chacun d’entre eux. En
faisant la moyenne des cinq valeurs de métrique, on obtient une meilleure estimation de celle-ci.
De plus, les cinq plis sont bien séparés. Cela veut dire que le modèle a été entraîné et testé cinq fois avec des données différentes à chaque fois. Cela favorise la généralisation du modèle pour de nouvelles données.
Une métrique moyenne, plus précise, permet de mieux optimiser
les paramètres du modèle entraîné.

Pourquoi ne pas utiliser la métrique moyenne comme valeur finale? La raison est simple. Bien que les
cinq plis soient bien séparés, chaque donnée est utilisée dans quatre entraînements. Il est donc possible
que cette surutilisation des mêmes données ait un effet sur le résultat final. C'est pourquoi on évalue les
performances finales avec l'ensemble de test qui contient de nouvelles données.
<!-- #endregion -->

<!-- #region id="d1738a83" -->
### <a id=raison-2>Raison 2</a>
<!-- #endregion -->

<!-- #region id="020e04e1" -->
En calculant l'écart-type des cinq valeurs de métrique, on obtient aussi une mesure de la robustesse de la
prédiction. Par exemple, un classificateur qui obtient un score moyen de $90~\%$ avec un écart-type de $7~\%$,
est moins fiable qu'un autre qui obtient un score de $89~\%$ avec un écart-type de $1~\%$. Contrairement
aux librairies R et SAS qui sont utilisées en statistiques, les fonctions de la librairie Scikit-learn ne fournissent pas les erreurs et les intervalles de confiances associés aux métriques calculées. Toutefois, la validation croisée permet de les estimer en analysant les valeurs générées par les plis. Ce n'est pas une faiblesse de Scikit-learn. Cette librairie a été conçue pour l’entraînement des modèles et la prédiction, pas pour faire de l'analyse statistique des paramètres. Ceci est plutôt la spécialité de logiciels comme R et SAS. Les buts sont différents, voilà tout.
<!-- #endregion -->

<!-- #region id="09ec53a0" -->
### <a id=raison-3>Raison 3</a>
<!-- #endregion -->

<!-- #region id="657f63a4" -->
La validation croisée gère mieux le problème des classes déséquilibrées. Supposons que l'ensemble d'entraînement
contienne 100 valeurs dont 10 appartiennent à la classe minoritaire. Séparer l'ensemble
d'entraînement en $80~\%$ train et $20~\%$ validation fera en sorte que l'ensemble de validation devrait contenir
en moyenne deux items de la classe minoritaire ($20~\%$ de 10 items). Il est probable qu'il n'y en ait qu'un ou même aucun. La mesure de la métrique de performance du classificateur serait trop imprécise. En utilisant cinq plis de $20~\%$ plutôt qu'un seul et en faisant la moyenne des résultats, on obtient une estimation plus réaliste de la métrique.
<!-- #endregion -->

<!-- #region id="71e5d2e7" -->
## <a id=entraînement-de-multiples-classificateurs-et-sélection-du-meilleur>Entraînement de multiples classificateurs et sélection du meilleur</a>
<!-- #endregion -->

<!-- #region id="c6e0787e" -->
La figure suivante montre l'approche couramment utilisée pour faire de la sélection de modèles. Chaque
modèle est entraîné comme précédemment avec les mêmes données d'entraînement. Le meilleur modèle est sélectionné selon la moyenne des résultats en validation croisée. Sa métrique de performance est ensuite
mesurée avec l'ensemble de test.

<!-- #endregion -->

<!-- #region id="9e987f01" -->
<div align="center">
    <img src= "../images/k-fold-cross-validation.jpeg"  width="700" />
    <div>
    <font size="0.5">Image Source: https://www.javierorraca.com/posts/2020-02-01-k-fold-cross-validation/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="d77d9563" -->
# <a id=un-exemple>Un exemple</a>
<!-- #endregion -->

<!-- #region id="54295aaa" -->
## <a id=lecture-et-préparation-des-données>Lecture et préparation des données</a>
<!-- #endregion -->

<!-- #region id="2fcd2ac0" -->
<div align="center">
    <img src= "../images/matrix-data-exchange.jpeg"  width="350" />
    <div>
    <font size="0.5">Image Source: https://pixabay.com/illustrations/matrix-network-data-exchange-1027571/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="31750fc8" -->
Nous allons nous servir du jeu de données sur le diabète utilisées dans un module précédent portant sur la classification par forêt aléatoire. 
Le jeu de données contient neuf variables (colonnes), la dernière Outcome contient la réponse binaire que l'on veut prédire, soit le sujet est atteint du diabète (« `'Outcome=1'` ») ou non (« `'Outcome=0'` »).
<!-- #endregion -->



In [ ]:
df = pd.read_csv("/pax/shared/GIF-U014/diabetes_nettoyée.csv")

X = df.drop(["Outcome"], axis=1)
y = df.Outcome



<!-- #region id="27a8a9ca" -->
### <a id=balancement-des-classes>Balancement des classes</a>
<!-- #endregion -->

<!-- #region id="08ef9d0e" -->
Les classes sont-elles balancées? Affichons le nombre de données pour chacune.
<!-- #endregion -->



In [ ]:
y.value_counts().plot(kind="bar").set_title("Diabetes Outcome")



<!-- #region id="2ce2ddf5" -->
La classe 0 comprend environ $65~\%$ des données; il y a un léger débalancement des données. Nous allons utiliser la
méthode d'imputation SMOTE afin d'éliminer ce déséquilibre. Le nombre d'éléments de la classe 1 sera augmenté
afin d'égaliser de la classe 0.
<!-- #endregion -->



In [ ]:
smote = SMOTE(sampling_strategy="minority")
X_sm, y_sm = smote.fit_resample(X, y)



<!-- #region id="d136c2d7" -->
Affichons à nouveau le nombre de données pour chaque classe.
<!-- #endregion -->



In [ ]:
y_sm.value_counts().plot(kind="bar").set_title("Diabetes Outcome")



<!-- #region id="045037b6" -->
Les classes sont maintenant balancées.

Générons les ensembles de données d'entraînement et de test. On utilise un
échantillonnage stratifié afin qu'il y ait les mêmes proportions de classes dans les deux ensembles.
Les données seront réparties comme suit: train ($80~\%$) et test ($20~\%$).
<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_sm, y_sm, test_size=0.2, shuffle=True, stratify=y_sm, random_state=seed
)



<!-- #region id="1da818f8" -->
### <a id=normalisation-des-données>Normalisation des données</a>
<!-- #endregion -->

<!-- #region id="e91f051b" -->
Les paramètres de la fonction de normalisation doivent être calculés à partir des données
d'entraînement uniquement. La même fonction est ensuite appliquée aux deux ensembles de données.
<!-- #endregion -->



In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)



<!-- #region id="d8badfc7" -->
## <a id=classification-par-forêt-aléatoire-en-utilisant-les-paramètres-par-défaut>Classification par forêt aléatoire en utilisant les paramètres par défaut</a>
<!-- #endregion -->

<!-- #region id="2cefe227" -->
Dans ce premier exemple, on entraîne le classificateur tel quel. C'est-à-dire, sans ajuster ses hyperparamètres
afin de maximiser ses performances en classification.

Chaque entraînement produira cinq valeurs de la métrique de performance utilisée, dans ce cas-ci, l'exactitude.
<!-- #endregion -->



In [ ]:
rf = RandomForestClassifier(random_state=42)
kf = KFold(n_splits=5)
result = cross_val_score(rf, X_train_s, y_train, cv=kf)


In [ ]:
print("Exactitude pour chaque pli: {}".format(result))
print("Exactitude moyenne: %0.3f" % (result.mean()))



<!-- #region id="5a13e452" -->
Selon ce résultat, le classificateur atteint une exactitude de $80,4~\%$. Est-ce que l'apprentissage s'est bien passé?
Y a-t-il surapprentissage? Pour le savoir on va prédire la réponse $y$ pour chaque ensemble de données, soit
celui d'entraînement, dont le classificateur a vu les données, et celui de test dont il n'a jamais vu les données.
<!-- #endregion -->



In [ ]:
# Entraînement avec toutes les données d'entraînement (sans pli)
model = rf.fit(X_train_s, y_train)

# Prédiction des réponses y pour chaque ensemble
y_pred_train = model.predict(X_train_s)
y_pred_test = model.predict(X_test_s)

print(
    "Exactitude sur les données d'entraînement: %0.1f %%"
    % (100 * accuracy_score(y_train, y_pred_train))
)
print(
    "Exactitude sur les données de test: %0.1f %%"
    % (100 * accuracy_score(y_test, y_pred_test))
)



<!-- #region id="b7f74ace" -->
<div align="center">
    <img src= "../images/man-pointing-silhouette.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: https://publicdomainpictures.net/en/view-image.php?image=74319&picture=man-pointing-silhouette&large=1</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="491ab678" -->
L'exactitude de $100~\%$ pour les données d'entraînement montre que le classificateur les a mémorisées;
il y a surapprentissage. D'ailleurs, la grande différence entre les deux résultats ($82~\%$ *versus* $100~\%$) est
un indicateur de surapprentissage.

Cela montre que les valeurs par défaut des classificateurs ne sont pas toujours les meilleures.
<!-- #endregion -->

<!-- #region id="1d02c818" -->
## <a id=optimisation-des-hyperparamètres-du-classificateur-par-forêt-aléatoire>Optimisation des hyperparamètres du classificateur par forêt aléatoire</a>
<!-- #endregion -->

<!-- #region id="2dd11b29" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/target-idea.jpeg"  width="350" />
    <div>
    <font size="0.5">Image Source: https://www.wallpaperflare.com/target-business-idea-growth-business-idea-concept-planning-wallpaper-aoydx</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="0e396051" -->
Quelles sont les valeurs des hyperparamètres permettant d'obtenir les meilleures performances en classification?
Pour le découvrir, nous allons effectuer une série d'entraînements en utilisant des hyperparamètres répartis
aux extrémités d'une grille de recherche. Un classificateur sera entraîné pour chaque point sur la grille et
le classificateur optimal sera déterminé en sélectionnant le meilleur parmi eux. Chaque hyperparamètre est
un « bouton » que l'on peut ajuster afin d'optimiser le résultat final. Le nombre de possibilités
augmente rapidement avec le nombre d'hyperparamètres.

Dans ce qui suit, nous allons nous concentrer sur trois d'entre eux:

- `n_estimators`,
- `max_depth`,
- `criterion`.


Le nombre total de combinaisons d'hyperparamètres est de $3 x 3 x 3 = 27$.
<!-- #endregion -->



In [ ]:
param_grid = {
    "n_estimators": [125, 150, 175],
    "max_depth": [1, 3, 5],
    "min_samples_split": [2, 3, 4],
}



<!-- #region id="684b6c8c" -->
Pour chaque ensemble d'hyperparamètres sur la grille de recherche, un classificateur sera entraîné et
ses performances seront évaluées en utilisant une validation croisée à cinq plis. Puisque chaque
modèle sera entraîné cinq fois, un total de $5 x 27 = 135$ modèles seront entraînés.

Puisque les classes sont maintenant balancées, chaque classificateur sera entraîné afin de maximiser l'exactitude (*Accuracy*).

> À noter que la fonction `GridSearchCV` assigne les hyperparamètres optimaux au classificateur de départ lorsque *refit=True*.
Ainsi, après avoir entraîné chaque modèle et isolé le meilleur, celui-ci est réentraîné en utilisant l'ensemble des données d'entraînement (c.-à-d. sans validation croisée).
<!-- #endregion -->



In [ ]:
rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    refit=True,
    n_jobs=2,
    verbose=2,
    scoring="accuracy",
)


In [ ]:
# Entraînement de tous les modèles sur la grille d'hyperparamètres

grid_search.fit(X_train_s, y_train)


In [ ]:
# Affichage du meilleur choix d'hyperparamètres

grid_search.best_params_



<!-- #region id="3cbe8c0f" -->
Refaisons le test précédent pour vérifier s'il y a surapprentissage. On doit prédire la réponse y en utilisant toutes les données d'entraînement et de test.
<!-- #endregion -->



In [ ]:
# Prédictions pour l'ensemble d'entraînement
pred_train = grid_search.predict(X_train_s)

# Prédictions pour l'ensemble de test
pred_test = grid_search.predict(X_test_s)


In [ ]:
# Affichage des valeurs d'exactitude pour les deux ensembles

print(
    "Exactitude sur les données d'entraînement: %0.1f %%"
    % (100 * accuracy_score(y_train, pred_train))
)

print(
    "Exactitude sur les données de test: %0.1f %%\n\n"
    % (100 * accuracy_score(y_test, pred_test))
)



<!-- #region id="9238f182" -->
<div align="center">
    <img src= "../images/man-pointing-silhouette.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: https://publicdomainpictures.net/en/view-image.php?image=74319&picture=man-pointing-silhouette&large=1</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="7b566a73" -->
Il n'y a plus de surapprentissage puisque les deux valeurs d'exactitude sont rapprochées et inférieures à $100~\%$!
De plus, l'exactitude sur les données de test ($81,0~\%$) est assez près de celle de $82,0~\%$ mesurée 
dans la section précédente.
<!-- #endregion -->

<!-- #region id="0cde5830" -->
Le rapport de classification fournit une comparaison plus fine des performances en test.
<!-- #endregion -->



In [ ]:
print(classification_report(y_test, pred_test))



<!-- #region id="e97aceea" -->
On obtient une exactitude (*Accuracy*) de $81~\%$. Ainsi $81~\%$ des prédictions (sujets avec ou sans diabète) sont correctes.

La précision est la proportion des résultats positifs qui correspondent réellement à des sujets diabétiques: $76\%$.

Le rappel (*Recall*) est la proportion des sujets diabétiques détectés : $90~\%$.
<!-- #endregion -->

<!-- #region id="e4eedf43" -->
# <a id=comment-choisir-son-modèle-final>Comment choisir son modèle final?</a>
<!-- #endregion -->

<!-- #region id="9aa18315" -->
<div align="center">
    <img src= "../images/model-choice.jpeg"  width="700" />
    <div>
    <font size="0.5">Image Source: https://www.javierorraca.com/posts/2020-02-01-k-fold-cross-validation/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="dda154ab" -->
Le classificateur par forêt aléatoire est-il le meilleur? Il est utile de comparer plusieurs classificateurs
entre eux et de choisir le meilleur. En pratique, on le fait souvent pour une dizaine de classificateurs ou plus.

Dans ce qui suit, nous allons comparer deux classificateurs; la forêt aléatoire et la régression logistique.

Liste des classificateurs à comparer:
<!-- #endregion -->



In [ ]:
classifiers = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(),
}



<!-- #region id="5ee826c6" -->
Liste des grilles de paramètres pour chacun. Dans cet exemple, on va réutiliser les
paramètres optimaux pour la forêt aléatoire. Toutefois, rien n'empêche d'utiliser la grille
de recherche précédente et refaire l'optimisation.

La classe `LogisticRegression` a deux importants paramètres à optimiser. Soit la méthode de régularisation
et le paramètre de régularisation mentionné dans le module sur la régularisation.
<!-- #endregion -->



In [ ]:
best = grid_search.best_params_
rf_param_grid = dict(map(lambda item: (item[0], [item[1]]), best.items()))

lr_param_grid = {"penalty": ["none", "l2"], "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

param_grid = {"Random Forest": rf_param_grid, "LogisticRegression": lr_param_grid}



<!-- #region id="fa2dfcfb" -->
## <a id=optimisation-sur-grille-et-affichage-des-résultats>Optimisation sur grille et affichage des résultats</a>
<!-- #endregion -->

<!-- #region id="26b57ac7" -->
Pour chaque classificateur, on affiche les valeurs d'exactitudes en entraînement et en test afin de déterminer s'il est en surapprentissage. Lorsque c'est le cas, il faut affiner les grilles de paramètres correspondantes.
<!-- #endregion -->



In [ ]:
warnings.filterwarnings("ignore")

head = list(classifiers.items())
best_model = []
for name, clf in head:
    params = param_grid[name]
    grid_search = GridSearchCV(
        estimator=clf,
        param_grid=params,
        cv=5,
        refit=True,
        n_jobs=2,
        verbose=2,
        scoring="accuracy",
    )
    grid_search.fit(X_train_s, y_train)

    score_train = grid_search.score(X_train_s, y_train)
    score_test = grid_search.score(X_test_s, y_test)

    best_model.append(grid_search)
    print("{:<15}:".format(name))
    print(
        "\tExactitude sur les données d'entraînement : %0.1f %%" % (100 * score_train)
    )
    print("\tExactitude sur les données de test : %0.1f %%" % (100 * score_test))

    print("\tParamètres optimaux: %s\n" % str(grid_search.best_params_))



<!-- #region id="ae87ac54" -->
Le meilleur classificateur demeure la forêt aléatoire avec une exactitude de $81,0~\%$.

Notez qu'aucun des classificateurs n’est en surapprentissage puisque les valeurs d'exactitudes en entraînement
et en test sont similaires et qu'aucune valeur n’atteint $100~\%$.
<!-- #endregion -->
